This code allows you to put in three words of your choosing and it will try to solve Semantle for you.

In [ ]:
import numpy as np
import os
import re
import random
from scipy.spatial.distance import cdist 
from sklearn.preprocessing import normalize
from gensim import models
from collections import defaultdict


## Load in the language model

In [ ]:
w2v = models.KeyedVectors.load("word2vec.model")

In [ ]:
lower_ascii = re.compile(r"^[a-z]+$")

In [ ]:
words = w2v.index_to_key

# semantle uses only lowercase ascii
words = [w for w in words if lower_ascii.match(w)]


Now we create a matrix of all the word vectors that go with these words and we'll renormalize.

In [ ]:
vecs = []

for word in words : 
    vecs.append(w2v[word])

vec_mat = np.vstack(vecs)
# Adapted from https://github.com/manimino/semantle-crab/blob/main/notebooks/crab.ipynb

In [ ]:
vec_mat = normalize(vec_mat,axis=1)

In [ ]:
vec_mat.shape

## Work with distances

In [ ]:
# Both of these functions build off of this: https://github.com/manimino/semantle-crab/blob/main/notebooks/crab.ipynb
# which has the correct semantle distance function

def get_all_dists(target,word_list, matrix):
    idx = word_list.index(target)
    vec = np.array([matrix[idx, :]])
    dists = cdist(vec, matrix, metric='cosine')
    semantle_scores = np.round((1-dists)*100, 1)[0]
    score_list = semantle_scores.tolist()
    return(score_list)


def pair_dist(word_1, word_2 ,word_list, matrix):
    idx_1 = word_list.index(word_1)
    idx_2 = word_list.index(word_2)
    vec_1 = np.array([matrix[idx_1, :]])
    vec_2 = np.array([matrix[idx_2, :]])
    dist = cdist(vec_1, vec_2, metric='cosine')
    semantle_score = np.round((1-dist)*100, 1)[0].tolist()
    return(semantle_score[0])


Put your first three guesses and their scores in the cell below.

In [ ]:
guesses = {
    'word1' : 0.0, # put the scores as values
    'word2' : 0.0,
    'word3' : 0.0
}

In [ ]:
scores = defaultdict(float)

for word,score in guesses.items() :
    all_scores = get_all_dists(word,words,vec_mat)
    
    for idx, score2 in enumerate(all_scores) : 
        scores[words[idx]] += abs(score2-score)
    
    
    
    

In [ ]:
print("Here are the top 5 guesses and their error.")

num_printed = 0
for word, error in sorted(scores.items(), key=lambda item: item[1]) :
    print(f"{word} has {error:.2f} error.")

    num_printed += 1
    if num_printed == 5 : 
        break
